## 函数与方法

python中函数与方法的区别是，方法的第一个参数是一个指向实例的指针`self`，而函数参数中不包含`self`指针。

函数中都包含一个`__get__()`方法，因此函数都可以被视为非数据型描述符。python正是通过非数据型描述符实现了将函数与一个实例绑定，从而使函数以一个方法的形式调用。

python正式通过函数的`__get__()`方法，将实例或类作为第一个参数传入函数中，从而实现函数与类的绑定。

In [30]:
class Foo:
    pass

def func(obj):
    print('func')
    
c = Foo()
func.__get__(c, Foo)

<bound method func of <__main__.Foo object at 0x0345D410>>

In [32]:
func.__get__(c, Foo)()

func


首先定义类`Foo`与函数`func()`。

当需要将一个该函数与该类的一个实例`c`绑定时，函数`func()`参数列表中的第一个参数`obj`是必不可少的，该参数将在函数调用时接受一个实例`c`作为参数，其作用与实例方法中的`self`参数相同。

`func.__get__(c, Foo)`将返回一个绑定了实例`c`的绑定方法，通过`func.__get__(c, Foo)()`即可调用该方法。

其中函数的`__get__()`方法的原理如下所示。

In [33]:
class Function(object):
    def __get__(self, instance, objtype=None):
        if instance is None:
            return self
        return types.MethodType(self, instance)

In [5]:
class D:
    def f(self, x):
        return x

d = D()
# 直接通过类的__dict__访问方法不会调用__get__()方法
print(D.__dict__['f'])
# 类属性查找方式访问方法将调用__get__()方法
print(D.f)
# 实例属性查找方式访问方法将调用__get__()方法，返回一个bound method对象
print(d.f)
# bound method对象中的属性存储了对应的方法、绑定的实例与类
print(d.f.__func__)
print(d.f.__self__)
print(d.f.__class__)

<function D.f at 0x031A3C00>
<function D.f at 0x031A3C00>
<bound method D.f of <__main__.D object at 0x031CA870>>
<function D.f at 0x031A3C00>
<class 'method'>


## 静态方法与类方法

非数据型描述符的特性提供了将一个函数与实例绑定并转换为方法的功能。

函数中包含一个`__get__()`方法，使的函数能够在作为一个实例属性被访问时，可以转换成一个方法。如对于实例`obj`，调用`obj.f(*args)`被转换为调用`f(obj, *args)`，而对于类`Clazz`，调用`Clazz.f(*args)`被转换为调用`f(*args)`。

|方法类型|实例调用|类调用|
|-|-|-|
|函数|`f(obj, *args)`|`f(*args)`|
|静态方法|`f(*args)`|`f(*args)`|
|类方法|`f(type(obj), *args)`|`f(Clazz, *args`|

### 类方法

与实例方法类似，类方法也需要指定其第一个参数。不同的是，实例方法第一个参数`self`指向的是一个实例，而类方法的第一个参数`cls`指向的是类本身。

类方法主要应用于需要类的引用，但又不需要相关实例数据的情况，多用于工厂方法。

In [13]:
class ClsMethodClazz:
    def func(cls, x):
        return cls.__name__, x
    func = classmethod(func)
    
print(ClsMethodClazz.func(3))
print(ClsMethodClazz().func(3))

('ClsMethodClazz', 3)
('ClsMethodClazz', 3)


其中`classmethod()`实现原理如下所示。

In [ ]:
class ClassMethod:
    def __init__(self, f):
        self.f = f
        
    def __get__(self, obj, cls=None):
        if cls is None:
            cls = type(obj)
        def newfunc(*args):
            return self.f(cls, *args)
        return newfunc

### 静态方法

静态函数是指函数参数中不包含指向实例的`self`参数，也不包含指向类的`cls`参数。

调用静态方法`func()`时，对于类`Clazz`与实例`obj`，分别调用的是`object.__getattribute__(Clazz, 'func')`与`object.__getattribute__(obj, 'func')`。因此通过实例或类调用静态方法是相同的。

In [11]:
class StaticMethodClazz:
    def func(x):
        return x
    func = staticmethod(func)

print(StaticMethodClazz.func(3))
print(StaticMethodClazz().func(3))

3
3


其中`staticmethod()`实现原理如下所示。

In [ ]:
class StaticMethod:
    def __init__(self, f):
        self.f = f
    
    def __get__(self, obj, objtype=None):
        return self.f